##Instalaciones

In [ ]:
!pip install bertopic
!pip install stopwords
!pip install nltk
!pip install unidecode
!pip install rake_new2 -q
!pip install accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.2 MB/s eta 0:00:00
  Preparing metadata (s

In [ ]:
from bertopic import BERTopic
import unidecode
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import transformers
from torch import bfloat16
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords
from umap import UMAP
from hdbscan import HDBSCAN
from rake_new2 import Rake

nltk.download('punkt')
random_state=11

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

sheet_id = "1KCXRP2viXlMsf3u8nP_7YcggwtKfAVZC"
sheet_name = "1500_respuestas_TIROALERTA"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
raw_data = pd.read_csv(url)
raw_data.columns

Index(['Started', 'Finished', '1. ¿Cuál es tu sexo?',
       '2. ¿Cuál es tu edad? - 1. Input - number',
       '3. ¿Cuál es tu profesión o a qué te dedicas? - 1. Input - text',
       '4. ¿Qué has intentado hacer para trabajar y conectar con los ángeles y amigos del cielo? - 1. Input - textarea',
       '5. ¿Cuál ha sido tu mayor desafío al intentar trabajar y conectar con los ángeles y amigos del cielo? - 1. Input - textarea',
       '6. Si pudieras tener una maestría, mentoría o curso donde se te entregue el paso a paso para trabajar con los ángeles y sanarte a ti mismo o los demás, ¿Sería algo que te interesaría?',
       '7. Participar en un grupo, donde haya más personas que trabajen con los ángeles, ¿Sería algo que te interesaría?',
       '8. Si estuvieras compartiendo un café con un terapeuta angelical, ¿Cuál sería la pregunta más importantes que le pedirías que te respondiera? - 1. Input - textarea',
       '9. ¿Qué no podría faltar en una maestría angelical, para que este pr

## Exploracion de datos

In [ ]:
#Seleccionando la columna
df_preguntas_orgn = raw_data.iloc[:,9]
#drop na
df_preguntas_orgn.dropna(inplace=True)
#a minusculas
df_preguntas_orgn = df_preguntas_orgn.apply(lambda x: " ".join(x.lower() for x in x.split()))
#estadisticas de respuestas
print("Max de caracteres x comentario: ", max(df_preguntas_orgn.str.len()))
print("Max de palabras x comentario: ", max(df_preguntas_orgn.str.split().str.len()))
df_preguntas_tildes = df_preguntas_orgn.apply(unidecode.unidecode)
df_preguntas_tildes.head(10)

Max de caracteres x comentario:  609
Max de palabras x comentario:  116


0                       cual es mi mision en esta vida
1    que debo hacer para estar en contacto con ello...
2       me pueden sanar de cualquier enfermedad fisica
3                                 como trabajar el ego
4                           cual es mi mision de vida?
5    si jehova es quien yo creo, el dios todopodero...
6                    cual sera nuestra proxima mision?
7    como aprender a conectar bien la energia angel...
8    si un dolor en mi cuerpo es mio o necesito seg...
9       cual es mi mision. como adquirir mas sabiduria
Name: 8. Si estuvieras compartiendo un café con un terapeuta angelical, ¿Cuál sería la pregunta más importantes que le pedirías que te respondiera? - 1. Input - textarea, dtype: object

##Entrenando embeddings y clasterizando/xlm-r-bert-base-nli-stsb-mean-tokens

###HDBSCAN

In [ ]:
# Hiperparametros estandares - Reduccion de dimencionalidad
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.1, metric='cosine')

# Hiperparametros estandares - Clusterizacion con HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                 n_gram_range=(1, 6),
                 hdbscan_model = hdbscan_model,
                 umap_model = umap_model,
                 min_topic_size=3,
                 nr_topics=30,
                 calculate_probabilities= True,
                 verbose=True )


In [ ]:
topics, probabilities = model.fit_transform(df_preguntas_tildes)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:23:19,998 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:23:27,439 - BERTopic - Reduced dimensionality
2023-10-03 23:23:27,608 - BERTopic - Clustered reduced embeddings
2023-10-03 23:23:28,423 - BERTopic - Reduced number of topics from 37 to 30


In [ ]:
frequency=model.get_topic_info()
frequency

,Topic,Count,Name,Representation,Representative_Docs
0,-1,338,-1_como_que_para_mi,"[como, que, para, mi, de, en, puedo, lo, me, es]","[cual es mi proposito., como hacer para ser ma..."
1,0,481,0_angeles_los_los angeles_con,"[angeles, los, los angeles, con, que, como, de...",[como lograr conectar con los angeles? si pued...
2,1,100,1_como_mensajes_ellos_con ellos,"[como, mensajes, ellos, con ellos, con ellos c...",[como me comunico efectivamente con ellos y co...
3,2,66,2_porque_que_no_de,"[porque, que, no, de, me, en, el, la, mi, para]",[como dominar un ego tan fuerte que ese si es ...
4,3,58,3_no_no se_se_se no,"[no, no se, se, se no, se no se, no se no, no ...","[no se, no se, no se]"
5,4,55,4_mision_mi mision_cual es mi_cual es,"[mision, mi mision, cual es mi, cual es, es mi...","[cual es mi mision, cual es mi mision, cual es..."
6,5,40,5_que_como_senales_hacer,"[que, como, senales, hacer, para, en, las sena...",[la pregunta mas importante que le pediria que...
7,6,36,6_espiritual_guia_mis guias_como,"[espiritual, guia, mis guias, como, guias, esp...",[como puedo conectar con mis guias espirituale...
8,7,28,7_mejor_para_que_ser mejor,"[mejor, para, que, ser mejor, ser, mejorar, me...","[que es lo que tengo que ver, sanar con respec..."
9,8,27,8_sanar_sanarme_como sanar_como,"[sanar, sanarme, como sanar, como, sanacion, s...",[que me diga como sanar mas rapido a las perso...


In [ ]:
model.get_topic(0)

[('angeles', 0.018626364681867333),
 ('los', 0.017258809942411853),
 ('los angeles', 0.016218211245294878),
 ('con', 0.012997545284668852),
 ('que', 0.012946859426723988),
 ('como', 0.01273581510098856),
 ('de', 0.011245506116863103),
 ('con los', 0.009699697825780555),
 ('con los angeles', 0.009691569274498761),
 ('angel', 0.0087339867201463)]

In [ ]:
model.get_document_info(df_preguntas_tildes).head(5)


,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,cual es mi mision en esta vida,4,4_mision_mi mision_cual es mi_cual es,"[mision, mi mision, cual es mi, cual es, es mi...","[cual es mi mision, cual es mi mision, cual es...",mision - mi mision - cual es mi - cual es - es...,1.000000,False
1,que debo hacer para estar en contacto con ello...,1,1_como_mensajes_ellos_con ellos,"[como, mensajes, ellos, con ellos, con ellos c...",[como me comunico efectivamente con ellos y co...,como - mensajes - ellos - con ellos - con ello...,0.314947,True
2,me pueden sanar de cualquier enfermedad fisica,8,8_sanar_sanarme_como sanar_como,"[sanar, sanarme, como sanar, como, sanacion, s...",[que me diga como sanar mas rapido a las perso...,sanar - sanarme - como sanar - como - sanacion...,0.358196,False
3,como trabajar el ego,-1,-1_como_que_para_mi,"[como, que, para, mi, de, en, puedo, lo, me, es]","[cual es mi proposito., como hacer para ser ma...",como - que - para - mi - de - en - puedo - lo ...,0.586988,False
4,cual es mi mision de vida?,16,16_cual_cual es_vida_cual es mi,"[cual, cual es, vida, cual es mi, es mi, vida ...",[cual es mi guion de vida ? cual es mi mision ...,cual - cual es - vida - cual es mi - es mi - v...,1.000000,True


###K-means

In [ ]:
cluster_model = KMeans(n_clusters=50)

In [ ]:
k_means_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                         hdbscan_model=cluster_model,
                          umap_model = umap_model,
                          n_gram_range=(1, 6),
                          min_topic_size=3,
                          nr_topics=30,
                          calculate_probabilities= True,
                          verbose=True )

In [ ]:
topics_k, probabilities_k = k_means_model.fit_transform(df_preguntas_tildes)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:23:58,564 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:24:06,642 - BERTopic - Reduced dimensionality
2023-10-03 23:24:06,855 - BERTopic - Clustered reduced embeddings
2023-10-03 23:24:07,647 - BERTopic - Reduced number of topics from 50 to 30


In [ ]:
frequency=k_means_model.get_topic_info()
frequency

,Topic,Count,Name,Representation,Representative_Docs
0,0,312,0_angeles_los_los angeles_con,"[angeles, los, los angeles, con, como, con los...","[?como conectarme con los angeles?, como conec..."
1,1,110,1_que_como_para_el,"[que, como, para, el, me, de, mas, ellos, lo, la]",[voy por el camino indicado para pasar al sigu...
2,2,86,2_que_no_de_los,"[que, no, de, los, angeles, los angeles, me, s...",[siento que entre mas conozco sobre los apoyos...
3,3,74,3_como_ellos_con ellos_para,"[como, ellos, con ellos, para, ellos como, con...",[saber ver y / o escuchar las senales! como co...
4,4,69,4_mision_mi mision_cual es mi_cual es,"[mision, mi mision, cual es mi, cual es, es mi...","[cual es mi mision, cual es mi mision, cual es..."
5,5,64,5_angel_mi angel_mi_angel de,"[angel, mi angel, mi, angel de, angel de la, g...","[quien es mi angel de la guarda, ?quien es mi ..."
6,6,64,6_sanar_como_salud_personas,"[sanar, como, salud, personas, para, mi, sanar...",[no se me ocurre ahora pero yo creo que seria ...
7,7,60,7_mensajes_como_que_ellos,"[mensajes, como, que, ellos, senales, me, los ...",[como poder entender o tener mayor claridad en...
8,8,58,8_vida_mi_mision_cual,"[vida, mi, mision, cual, de vida, mi mision, c...","[cual es mi mision en esta vida ?, cual es mi ..."
9,9,58,9_que_no_me_como,"[que, no, me, como, porque, se, para, de, no s...",[?de ser posible mediante una respuesta detall...


###Resumen
no me convense el procesamiento en la etapa de **c-TF-IDF**, porque los label que devuelve no son muy informativas, a veces sus variaciones muy cercanas repiten entre los clasters. Quiero quitar los stopwords justo antes de esta fase,dejandolas en la fase anterior de embeddings porque son importantes para entendimieto del contextp de las frases y textos en general.

##Quitando stop-words

In [ ]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Get the Spanish stopwords
spanish_stopwords = stopwords.words('spanish')
stop = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop=stop + ['si', 'tan', 'va']

###HDBSCAN

In [ ]:
 #eliminando stopwords antes de tolenizar de palabras
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)

In [ ]:
model_stop = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                      hdbscan_model = hdbscan_model,
                      umap_model = umap_model,
                      min_topic_size=3,
                      nr_topics=30,
                      vectorizer_model=vectorizer_model,
                      calculate_probabilities= True,
                      verbose=True )

topics, probabilities = model_stop.fit_transform(df_preguntas_tildes)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:25:52,253 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:25:58,292 - BERTopic - Reduced dimensionality
2023-10-03 23:25:58,466 - BERTopic - Clustered reduced embeddings
2023-10-03 23:25:58,691 - BERTopic - Reduced number of topics from 36 to 30


In [ ]:
Topics_hdbscan_stop=model_stop.get_topic_info()
Topics_hdbscan_stop

,Topic,Count,Name,Representation,Representative_Docs
0,-1,279,-1_vida_puedo_saber_proposito,"[vida, puedo, saber, proposito, ayudar, mision...","[mi familia es muy importante ,soy madre cabez..."
1,0,481,0_angeles_angel_puedo_conectar,"[angeles, angel, puedo, conectar, angelical, g...","[cual es mi angel., cual es mi angel ?, como c..."
2,1,64,1_sanar_salud_sanarme_personas,"[sanar, salud, sanarme, personas, ayudar, sana...",[como sanarme y ayudar a sanar a otras persona...
3,2,59,2_conexion_conectar_conectarme_hacer,"[conexion, conectar, conectarme, hacer, presen...","[como hacer la conexion?, como estan mis papas..."
4,3,56,3_mision_vida mision_vida_mision vida,"[mision, vida mision, vida, mision vida, misio...","[mi mision en esta vida, cual es mi mision de ..."
5,4,53,4_siento_vida_hacer_pareja,"[siento, vida, hacer, pareja, dinero, mal, cue...",[porque siento que estoy bloqueada o estancada...
6,5,48,5_mensajes_escucharlos_poder_estan,"[mensajes, escucharlos, poder, estan, interpre...","[como hablar con ellos, pedirles senales y men..."
7,6,46,6_paz_amor_feliz_paz interior,"[paz, amor, feliz, paz interior, interior, per...",[como puedo sentirme feliz? llegara la prosper...
8,7,41,7_camino_mas alla_alla_empiezo,"[camino, mas alla, alla, empiezo, bloqueos, ma...","[como es el mas alla, al final de tanto tiempo..."
9,8,30,8_ayudar estan_puedo_ayudar_ayudarme,"[ayudar estan, puedo, ayudar, ayudarme, puedo ...","[como puedo ayudar a su causa?, me gustaria qu..."


In [ ]:
Topics_hdbscan_stop['Representative_Docs'][2] #Vemamos las respuestas representativas del claster 2

['como sanarme y ayudar a sanar a otras personas. porque a mi se me acercan muchas personas con grandes dificultades de vida.',
 'como ayudar a las personas, como sanar',
 'como sanar a otras personas']

In [ ]:
model.get_topic(1)

[('como', 0.02778400884641204),
 ('mensajes', 0.018162502712256813),
 ('ellos', 0.017233081923040097),
 ('con ellos', 0.01612236763883913),
 ('con ellos como', 0.015663766812923613),
 ('ellos como', 0.014947365947459282),
 ('que', 0.013633831818417404),
 ('con', 0.011834204822924974),
 ('para', 0.011599125975675735),
 ('me', 0.010207972409674318)]

In [ ]:
model.visualize_barchart()


###K-means

In [ ]:
cluster_model = KMeans(n_clusters=50)

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)

k_means_stop = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                        umap_model = umap_model,
                        hdbscan_model=cluster_model,
                        min_topic_size=3,
                        nr_topics=30,
                        vectorizer_model=vectorizer_model,
                        calculate_probabilities= True,
                        verbose=True )


In [ ]:
topics2, probabilities2 = k_means_stop.fit_transform(df_preguntas_tildes)
Topics_k_means_stop=k_means_stop.get_topic_info()
Topics_k_means_stop

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:26:51,644 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:26:59,730 - BERTopic - Reduced dimensionality
2023-10-03 23:26:59,948 - BERTopic - Clustered reduced embeddings
2023-10-03 23:27:00,206 - BERTopic - Reduced number of topics from 50 to 30


,Topic,Count,Name,Representation,Representative_Docs
0,0,309,0_angeles_conectar_conectar angeles_angel,"[angeles, conectar, conectar angeles, angel, p...","[como conectar con los angeles, como conectar ..."
1,1,133,1_conexion_conectar_conectarme_hacer,"[conexion, conectar, conectarme, hacer, mas, s...",[como puedo utilizar los codigos sagrados corr...
2,2,86,2_angeles_angel_siento_puedo,"[angeles, angel, siento, puedo, quiero, solo, ...","[me gustaria seguir con mi mision de vida ,sie..."
3,3,70,3_angel_guarda_angel guarda_guardian,"[angel, guarda, angel guarda, guardian, angel ...","[quien es mi angel de la guarda, quien es mi a..."
4,4,67,4_mision_vida_vida mision_mision vida,"[mision, vida, vida mision, mision vida, propo...","[cual es mi mision para esta vida, cual es la ..."
5,5,61,5_camino_camino correcto_correcto_debo,"[camino, camino correcto, correcto, debo, siem...","[mas que una pregunta, que me adentrara mas en..."
6,6,60,6_siento_vida_situacion_hacer,"[siento, vida, situacion, hacer, tenia, situac...",[porque siento que estoy bloqueada o estancada...
7,7,57,7_vida_mision_mision vida_cumplir,"[vida, mision, mision vida, cumplir, proposito...","[cual es mi mision en esta vida ?, cual es mi ..."
8,8,50,8_mensajes_escucharlos_poder_interpretar,"[mensajes, escucharlos, poder, interpretar, en...",[le pediria que me ayude a poder escucharlos o...
9,9,49,9_sanar_salud_personas_sanarme,"[sanar, salud, personas, sanarme, sanacion, pu...","[como ayudar a las personas, como sanar, si es..."


###Resumen
quitar los stopwords ha mejorado la persepcion humana de las palabras claves con los que se caracterizaban los clusters detectados. Tambien he experimentdado con parametros que regulan cantidad de palabras en un token, numero minimo de docuemntos por tema y numero maximo de clusters. Pero esto no ha dado mejoras significativas en cuanto a la calidad de clusterizacion y calidad de representacion de los temas con sus palabras claves a mi opinion.
Y lo ultimo es muy importante para que al final la calidad de los resultados sea aceptable para la meta.
La mejora en la representacion de los clusters puede agregar mucha calidad a nuestro resultado, porque ahora sigue siendo muy baja - con palabras que se repiten, que muchas veces no representan lo mas importante del cluster, que no ofrecen entendimeinto sobre que temas tratan los clusters detectados, es decir no tienen mucho sentido a veces.
Por tanto seguire experimentando con los modelos de representacion.

#parafrase-multilingual-mpnet-base-v2


In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "MMR": mmr,
}

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3))

hdb_repr_stop = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        hdbscan_model = hdbscan_model,
                        umap_model = umap_model,
                         vectorizer_model=vectorizer_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=30,
                        calculate_probabilities= True,
                        verbose=True )


In [ ]:
topics, probabilities = hdb_repr_stop.fit_transform(df_preguntas_tildes)
topics_hdb_repr_stop=hdb_repr_stop.get_topic_info()
topics_hdb_repr_stop

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:36:02,449 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:36:10,524 - BERTopic - Reduced dimensionality
2023-10-03 23:36:10,813 - BERTopic - Clustered reduced embeddings
2023-10-03 23:36:19,035 - BERTopic - Reduced number of topics from 59 to 30


,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,464,-1_que_de_como_los,"[que, de, como, los, mi, me, angeles, en, para...","[mis angeles, angeles como, con los angeles, a...","[que, de, como, los, mi, me, angeles, en, para...",[siento que entre mas conozco sobre los apoyos...
1,0,266,0_angeles_los angeles_los_con,"[angeles, los angeles, los, con, con los angel...","[los angeles como, que los angeles, con los an...","[angeles, los angeles, los, con, con los angel...","[como conectar con los angeles, como conectar ..."
2,1,89,1_mision_cual es mi_es mi_mi mision,"[mision, cual es mi, es mi, mi mision, cual es...","[mi mision cual, es mi mision, mi mision en, m...","[mision, cual es mi, es mi, mi mision, cual es...","[cual es mi mision, cual es mi mision, cual es..."
3,2,80,2_con ellos_ellos_como_conexion,"[con ellos, ellos, como, conexion, con, ellos ...","[conectarme con ellos, conectar con ellos, par...","[con ellos, ellos, como, conexion, con, ellos ...",[como debo prepararme para ser un canal de.com...
4,3,66,3_como_estan_mensajes_los mensajes,"[como, estan, mensajes, los mensajes, que, ver...","[entender los mensajes, mensajes que ellos, lo...","[como, estan, mensajes, los mensajes, que, ver...","[como entender los mensajes, como saber que el..."
5,4,61,4_sanar_salud_personas_como,"[sanar, salud, personas, como, sanarme, sanar ...","[sanar otras personas, ayudar las personas, vi...","[sanar, salud, personas, como, sanarme, sanar ...","[como ayudar a las personas, como sanar, como ..."
6,5,55,5_espiritual_para_paz_como,"[espiritual, para, paz, como, que, mi, con, de...","[mis guias espirituales, guias espirituales, e...","[espiritual, para, paz, como, que, mi, con, de...",[como podria conectarme con los arcangeles mig...
7,6,52,6_camino_el camino_que_voy,"[camino, el camino, que, voy, el, camino que, ...","[camino que debo, camino correcto para, para l...","[camino, el camino, que, voy, el, camino que, ...",[si lo que estoy estudiando y tratando de hace...
8,7,41,7_no se_no_se no_se,"[no se, no, se no, se, no se no, se no se, me ...","[no se, no se en, no lo se, ocurre no se, segu...","[no se, no, se no, se, no se no, se no se, me ...","[no se, no se, no se]"
9,8,31,8_luz_de luz_seres de luz_seres de,"[luz, de luz, seres de luz, seres de, seres, d...","[seres de luz, ser de luz, de luz como, luz co...","[luz, de luz, seres de luz, seres de, seres, d...","[tener visiones con los seres de luz, como hac..."


###Quitando stopwards

####DBSCAN

In [ ]:


# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "MMR": mmr,
}

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)

hdb_repr = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                    hdbscan_model = hdbscan_model,
                    umap_model = umap_model,
                    vectorizer_model=vectorizer_model,
                      representation_model = representation_model,
                      min_topic_size=3,
                      nr_topics=30,
                      calculate_probabilities= True,
                      verbose=True )

topics, probabilities = hdb_repr.fit_transform(df_preguntas_tildes)
topics_hdb_repr=hdb_repr.get_topic_info()
topics_hdb_repr


Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:38:07,127 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:38:14,395 - BERTopic - Reduced dimensionality
2023-10-03 23:38:14,634 - BERTopic - Clustered reduced embeddings
2023-10-03 23:38:21,909 - BERTopic - Reduced number of topics from 45 to 30


,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,281,-1_vida_puedo_hacer_mas,"[vida, puedo, hacer, mas, proposito, ayudar, m...","[encontrar proposito vida, encontrar proposito...","[vida, puedo, hacer, mas, proposito, ayudar, m...",[hoy con 64 anos me siento un poco perdida por...
1,0,483,0_angeles_angel_puedo_conectar,"[angeles, angel, puedo, conectar, angelical, g...","[conectarme angeles, comunicarme angeles, cone...","[angeles, angel, puedo, conectar, angelical, g...","[como conectar con los angeles?, como conectar..."
2,1,86,1_mision_vida_mision vida_proposito,"[mision, vida, mision vida, proposito, vida mi...","[reconocer mision vida, vida mision vida, misi...","[mision, vida, mision vida, proposito, vida mi...","[cual es mi mision en esta vida, cual es mi mi..."
3,2,72,2_estan_mensajes_verlos_escucharlos,"[estan, mensajes, verlos, escucharlos, entende...","[entender mensajes, interpretar mensajes, perc...","[estan, mensajes, verlos, escucharlos, entende...",[mi mente preguntaria como saber si realmente ...
4,3,72,3_conexion_conectar_conectarme_hacer,"[conexion, conectar, conectarme, hacer, debo, ...","[lograr conexiones debo, logro conectar manera...","[conexion, conectar, conectarme, hacer, debo, ...",[presisamente si para lograr conectarme necesi...
5,4,62,4_espiritual_fe_paz_tener,"[espiritual, fe, paz, tener, espirituales, san...","[mas espiritual ayudar, guias espirituales, gu...","[espiritual, fe, paz, tener, espirituales, san...",[si yo como persona mayor podria llegar a cone...
6,5,56,5_sanar_personas_salud_sanarme,"[sanar, personas, salud, sanarme, sanacion, cu...","[poder sanar personas, sanar personas, pueda c...","[sanar, personas, salud, sanarme, sanacion, cu...",[si es posible sanar instantaneamente sanar a ...
7,6,39,6_ocurre_momento_momento ocurre_ocurre momento,"[ocurre, momento, momento ocurre, ocurre momen...","[ninguna particular supongo, ocurre ninguna pa...","[ocurre, momento, momento ocurre, ocurre momen...","[no se me ocurre por el momento, no se me ocur..."
8,7,35,7_mas alla_alla_dificil_preguntas,"[mas alla, alla, dificil, preguntas, mas, much...","[mas alla cuanto, momento dificil saber, nose ...","[mas alla, alla, dificil, preguntas, mas, much...","[que puedo hacker payar felicia dad?, no seria..."
9,8,31,8_luz_seres luz_seres_conectar seres luz,"[luz, seres luz, seres, conectar seres luz, co...","[conectar seres luz, percibe seres luz, comuni...","[luz, seres luz, seres, conectar seres luz, co...",[como poder conectar con mis seres de luz y pe...


####K-means (desde aqui no rerun)

In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "MMR": mmr,
}

In [ ]:
cluster_model = KMeans(n_clusters=50)

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)

k_means_repr_stop = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        vectorizer_model=vectorizer_model,
                        hdbscan_model=cluster_model,
                        umap_model = umap_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=30,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = k_means_repr_stop.fit_transform(df_preguntas_tildes)
Topics_k_means_repr_stop=k_means_repr_stop.get_topic_info()
Topics_k_means_repr_stop

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:39:02,181 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:39:09,800 - BERTopic - Reduced dimensionality
2023-10-03 23:39:10,007 - BERTopic - Clustered reduced embeddings
2023-10-03 23:39:18,969 - BERTopic - Reduced number of topics from 50 to 30


,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,466,0_angeles_angel_puedo_conectar,"[angeles, angel, puedo, conectar, angelical, g...","[angeles seres, conectarme angeles, angeles an...","[angeles, angel, puedo, conectar, angelical, g...","[cual es mi angel, como conectar con los angel..."
1,1,117,1_mision_vida_mision vida_proposito,"[mision, vida, mision vida, proposito, vida mi...","[vida mision vida, mision mision vida, encontr...","[mision, vida, mision vida, proposito, vida mi...","[mi mision en esta vida, cual es mi mision en ..."
2,2,77,2_camino_correcto_camino correcto_puedo,"[camino, correcto, camino correcto, puedo, hac...","[ser mejor persona, mejor puedo hacer, puedo h...","[camino, correcto, camino correcto, puedo, hac...",[como puedo descubrir mis talentos y mis dones...
3,3,73,3_hacer_conectarme_podrian_conectar,"[hacer, conectarme, podrian, conectar, comunic...","[poder conectarme, mejor forma contactarlos, p...","[hacer, conectarme, podrian, conectar, comunic...",[que podria hacer para elevar mi frecuencia y ...
4,4,71,4_sanar_salud_personas_sanarme,"[sanar, salud, personas, sanarme, sanacion, cu...","[sanar personas, poder ayudar sanacion, person...","[sanar, salud, personas, sanarme, sanacion, cu...",[si es posible sanar instantaneamente sanar a ...
5,5,63,5_mas_alla_ustedes_momento,"[mas, alla, ustedes, momento, mas alla, rapido...","[preguntara necesite momento, pueda ahi, pregu...","[mas, alla, ustedes, momento, mas alla, rapido...",[que hay alla donde ustedes estan y que posibi...
6,6,55,6_hijo_familia_hijos_murio,"[hijo, familia, hijos, murio, hijas, esposo, p...","[padres plano hijas, mundo padres, nietos logr...","[hijo, familia, hijos, murio, hijas, esposo, p...",[mis hijas andrea carolina torres pinzon y dan...
7,7,42,7_ocurre_momento_ocurre momento_momento ocurre,"[ocurre, momento, ocurre momento, momento ocur...","[ninguna particular supongo, ocurre ninguna pa...","[ocurre, momento, ocurre momento, momento ocur...","[no se me ocurre en el momento, no se me ocurr..."
8,8,40,8_espiritual_espirituales_mas espiritual_espir...,"[espiritual, espirituales, mas espiritual, esp...","[ser mas espiritual, espiritual ayudar, espiri...","[espiritual, espirituales, mas espiritual, esp...",[la mejor tecnica o forma para conectar con lo...
9,9,38,9_conexion_conectar_conectar puedo_hacer conexion,"[conexion, conectar, conectar puedo, hacer con...","[lograr conexion requiere, lograr conexion, lo...","[conexion, conectar, conectar puedo, hacer con...","[como hacer la conexion?, como puedo lograr es..."


###Resumen
Entre k-means y DBSCAN hay ligeras diferencias en las clases que agrupan, pero no diria que hay mucha. He revisado detenidamente los dos outputs y segun mi juicio, las palabras o frases claves que representan los clusters son parecidos, es decir los clasters han sido agrupados de manera parecida tamnbien.
los stopwords ayudan a tener mas sentido en las representaciones, aunque no parezcan luego al habla humano, dan mejor idea del tema de claster.

## 1-4 n-gramas

###DBSCAN

In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "MMR": mmr,
}

vectorizer_model = CountVectorizer(ngram_range=(1, 4), stop_words=stop)

hdb_repr_gram = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        hdbscan_model = hdbscan_model,
                        umap_model = umap_model,
                        vectorizer_model=vectorizer_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=30,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = hdb_repr_gram.fit_transform(df_preguntas_tildes)
topics_hdb_repr_gram=hdb_repr_gram.get_topic_info()
topics_hdb_repr_gram

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:40:00,123 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:40:06,917 - BERTopic - Reduced dimensionality
2023-10-03 23:40:07,102 - BERTopic - Clustered reduced embeddings
2023-10-03 23:40:14,766 - BERTopic - Reduced number of topics from 43 to 30


,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,235,-1_puedo_mas_hacer_vida,"[puedo, mas, hacer, vida, saber, proposito, ay...","[sufrir, siento perdida, proposito vida, seres...","[puedo, mas, hacer, vida, saber, proposito, ay...",[querria saber si tengo las posibidades de pod...
1,0,482,0_angeles_angel_puedo_conectar,"[angeles, angel, puedo, conectar, angelical, g...","[conectarme angeles, conectar angeles, comunic...","[angeles, angel, puedo, conectar, angelical, g...","[como conectar con los angeles, como conectar ..."
2,1,95,1_mision_vida_mision vida_proposito,"[mision, vida, mision vida, proposito, vida mi...","[reconocer mision vida proposito, realizar pon...","[mision, vida, mision vida, proposito, vida mi...","[cual es mi mision de vida?, cual es mi mision..."
3,2,85,2_conexion_conectar_conectarme_hacer,"[conexion, conectar, conectarme, hacer, debo, ...","[lograr conexiones debo hacer, lograr conectar...","[conexion, conectar, conectarme, hacer, debo, ...",[presisamente si para lograr conectarme necesi...
4,3,69,3_mensajes_estan_verlos_escucharlos,"[mensajes, estan, verlos, escucharlos, poder, ...","[entender mensajes, mensajes siento, mensajes ...","[mensajes, estan, verlos, escucharlos, poder, ...",[le preguntaria como podria entender mas los m...
5,4,59,4_sanar_salud_personas_sanarme,"[sanar, salud, personas, sanarme, sanacion, cu...","[poder ayudar sanacion personas, poder sanar p...","[sanar, salud, personas, sanarme, sanacion, cu...",[como sanarme y ayudar a sanar a otras persona...
6,5,50,5_espiritual_paz_espirituales_mas espiritual,"[espiritual, paz, espirituales, mas espiritual...","[ser mas espiritual, espiritual ayudar, hacer ...","[espiritual, paz, espirituales, mas espiritual...",[la mejor tecnica o forma para conectar con lo...
7,6,44,6_ocurre_momento_momento ocurre_ocurre momento,"[ocurre, momento, momento ocurre, ocurre momen...","[ocurre ninguna particular supongo, ninguna pa...","[ocurre, momento, momento ocurre, ocurre momen...","[no se me ocurre en el momento, no se me ocurr..."
8,7,31,7_luz_seres luz_seres_conectar seres luz,"[luz, seres luz, seres, conectar seres luz, co...","[poder comunicarme seres luz, poder conectar s...","[luz, seres luz, seres, conectar seres luz, co...",[como poder conectar con mis seres de luz y pe...
9,8,30,8_mas alla_alla_preguntas_dificil,"[mas alla, alla, preguntas, dificil, mas, buen...","[verlo momento dificil saber, tendria verlo mo...","[mas alla, alla, preguntas, dificil, mas, buen...","[es dificil saber cual es la mas importante, n..."


### Split los documntos por signo interrogacion
La gipotesis: cada comentario que hace una persona originalmente se guarda en una celda, pero en este caso si tiene varias tematicas en su respuesta podrian ser vectorizadas como un solo tema. Y hacer el split por el signo de interrogacion puede ayudar a la mejor deteccion y representacion de temas. Quizas tendria sentido experimentar luego con split por signo de explamacion y/o el punto.

In [ ]:
#Split por signo interrogatorio
df_preguntas_sep = df_preguntas_tildes.str.split('?', expand = False).explode()
df_preguntas_sep.head()

0                       cual es mi mision en esta vida
1    que debo hacer para estar en contacto con ello...
2       me pueden sanar de cualquier enfermedad fisica
3                                 como trabajar el ego
4                            cual es mi mision de vida
Name: 8. Si estuvieras compartiendo un café con un terapeuta angelical, ¿Cuál sería la pregunta más importantes que le pedirías que te respondiera? - 1. Input - textarea, dtype: object

In [ ]:
df_preguntas_tildes.shape

(1469,)

In [ ]:
df_preguntas_sep.shape

(2002,)

In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "MMR": mmr,
}
vectorizer_model = CountVectorizer(ngram_range=(1, 4), stop_words=stop)

hdb_repr_split = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        hdbscan_model = hdbscan_model,
                        umap_model = umap_model,
                        vectorizer_model=vectorizer_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=30,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = hdb_repr_split.fit_transform(df_preguntas_sep)
topics_hdb_repr_split=hdb_repr_split.get_topic_info()
topics_hdb_repr_split

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2023-10-03 23:41:26,231 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:41:38,614 - BERTopic - Reduced dimensionality
2023-10-03 23:41:39,074 - BERTopic - Clustered reduced embeddings
2023-10-03 23:41:49,087 - BERTopic - Reduced number of topics from 67 to 30


,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,465,-1_angeles_puedo_vida_saber,"[angeles, puedo, vida, saber, angel, hacer, ma...","[comunicarme angeles, conectarme angeles, cone...","[angeles, puedo, vida, saber, angel, hacer, ma...",[la mejor tecnica o forma para conectar con lo...
1,0,413,0____,"[, , , , , , , , , ]","[, , , , , , , , , ]","[, , , , , , , , , ]","[, , ]"
2,1,297,1_angeles_angel_conectar_angelical,"[angeles, angel, conectar, angelical, puedo, g...","[conectar angeles, conectarme angeles, comunic...","[angeles, angel, conectar, angelical, puedo, g...","[como conectar con los angeles, como conectar ..."
3,2,124,2_mision_vida_mision vida_proposito,"[mision, vida, mision vida, proposito, vida mi...","[mision mision vida, vida mision vida, encontr...","[mision, vida, mision vida, proposito, vida mi...","[cual es mi mision de vida, cual es la mision ..."
4,3,97,3_sanar_personas_ayudar_salud,"[sanar, personas, ayudar, salud, sanarme, pued...","[personas poder ayudar sanacion, sanar persona...","[sanar, personas, ayudar, salud, sanarme, pued...",[ quizas me gustaria saber mas especificamente...
5,4,82,4_conexion_conectarme_conectar_hacer,"[conexion, conectarme, conectar, hacer, debo, ...","[llegar capaces conectar conectar, llegar capa...","[conexion, conectarme, conectar, hacer, debo, ...","[como hacer la conexion, como debo prepararme ..."
6,5,80,5_espiritual_fe_paz_ego,"[espiritual, fe, paz, ego, ayudar, mas, dios, ...","[mas espiritual tener, espiritualmente puedo, ...","[espiritual, fe, paz, ego, ayudar, mas, dios, ...",[como podria conectarme con los arcangeles mig...
7,6,63,6_mas_mas alla_alla_mas importante,"[mas, mas alla, alla, mas importante, pregunta...","[pregunta mas importante pediria, pediria resp...","[mas, mas alla, alla, mas importante, pregunta...","[no seria solo una, serian muchas preguntas ya..."
8,7,45,7_luz_seres luz_seres_conectar seres luz,"[luz, seres luz, seres, conectar seres luz, po...","[poder comunicarme seres luz, puedo conectar s...","[luz, seres luz, seres, conectar seres luz, po...",[como poder conectar con mis seres de luz y pe...
9,8,44,8_mensajes_escucharlos_entender_estan,"[mensajes, escucharlos, entender, estan, escuc...","[mensajes envian poder entender, personas pued...","[mensajes, escucharlos, entender, estan, escuc...",[le preguntaria como podria entender mas los m...


###Resumen
No noto (subjetivamente) mucha diferencia con el split ejecutado

##llama-2-7b-langchain-chat

Veamos que puede mejorar uno de los modelos generativos de state-of-the-art, elaborado por meta, entrenado en un corpus en español
Antes que nada, quantizamos el modelopara poder descargar y ejecutarla.

In [ ]:
#set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
  )

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, pipeline

model_id = "garrachonr/LlamaDos"


# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [ ]:
embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

  # KeyBERT
  keybert = KeyBERTInspired()
  # Text generation with Llama 2
  llama2 = TextGeneration(generator, prompt=prompt)

  # All representation models
  representation_model = {
      "KeyBERT": keybert,
      "Llama2": llama2
  }

  #Vectorizer model
  vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)


hdb_repr_gram = BERTopic(
                        # Sub-models
                        embedding_model=embedding_model,
                        vectorizer_model=vectorizer_model,
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        representation_model = representation_model,

                        # Hyperparameters
                        top_n_words=10,
                        min_topic_size=3,
                        nr_topics=30,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = hdb_repr_gram.fit_transform(df_preguntas_tildes)
topics_hdb_repr_gram=hdb_repr_gram.get_topic_info()
topics_hdb_repr_gram

#RAKE on top

In [ ]:
df_preguntas_sep.head()

0                       cual es mi mision en esta vida
1    que debo hacer para estar en contacto con ello...
2       me pueden sanar de cualquier enfermedad fisica
3                                 como trabajar el ego
4                            cual es mi mision de vida
Name: 8. Si estuvieras compartiendo un café con un terapeuta angelical, ¿Cuál sería la pregunta más importantes que le pedirías que te respondiera? - 1. Input - textarea, dtype: object

In [ ]:
#modelando los topicos para Rake con Bertopic-K-means

In [ ]:
stop=stop + [ 'podria', 'hacer', 'pues', 'pasa', 'gustaria', 'tambien', 'demas', 'ser', 'seria', 'serian']

In [ ]:
stop=stop + [ 'puedo', 'aun', 'sere', 'lograr', 'realizar', 'hace', 'solo', 'aca', 'hacia']

In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# All representation models
representation_model = {
    "KeyBERT": keybert
}
cluster_model = KMeans(n_clusters=50)

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stop)

k_means_repr_stop = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        vectorizer_model=vectorizer_model,
                        hdbscan_model=cluster_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=50,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = k_means_repr_stop.fit_transform(df_preguntas_tildes)
Topics_k_means_repr_stop=k_means_repr_stop.get_topic_info()
Topics_k_means_repr_stop

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-03 23:43:27,468 - BERTopic - Transformed documents to Embeddings
2023-10-03 23:43:35,991 - BERTopic - Reduced dimensionality
2023-10-03 23:43:36,593 - BERTopic - Clustered reduced embeddings
2023-10-03 23:43:39,926 - BERTopic - Reduced number of topics from 50 to 50


,Topic,Count,Name,Representation,KeyBERT,Representative_Docs
0,0,24,0_angeles_paso_preguntaria acerca_numeros,"[angeles, paso, preguntaria acerca, numeros, p...","[ensene conectarme angeles, comunicarme angele...",[mas bien en vez de pregunta le diria que si p...
1,1,19,1_sabiduria_obtener sabiduria_amor_obtener,"[sabiduria, obtener sabiduria, amor, obtener, ...","[obtener sabiduria alcanzar, obtener sabiduria...",[una sola no me bastaria.....pero preguntaria ...
2,2,27,2_ahora_claras ahora_carolina claras ahora_car...,"[ahora, claras ahora, carolina claras ahora, c...","[claras ahora preguntar, pregunta, pregunta cl...","[no se por ahora, no tengo una pregunta clara,..."
3,3,46,3_mision vida_mision_vida_vida mision,"[mision vida, mision, vida, vida mision, misio...","[mision objetivo vida, mision vida enfocarme, ...","[cual es mi mision en esta vida, cual es la mi..."
4,4,40,4_conectarme_elevar_vibracion_podrian,"[conectarme, elevar, vibracion, podrian, conec...","[poder conectarme, podrian ayudarme, interesa ...",[podrian ayudarme a ordenar mis conflictos int...
5,5,42,5_espiritual_espirituales_mas espiritual_espir...,"[espiritual, espirituales, mas espiritual, esp...","[guias espirituales, espiritual ayudar, guia e...",[la mejor tecnica o forma para conectar con lo...
6,6,22,6_mas alla_alla_ummm sanaruas_mas alla mas,"[mas alla, alla, ummm sanaruas, mas alla mas, ...","[mas alla cuanto, hcerlo huy mas, mas alla, al...","[que hay mas alla, como es el mas alla, como s..."
7,7,25,7_ayudar_ayudarme_personas_ayudar personas,"[ayudar, ayudarme, personas, ayudar personas, ...","[entender ayudar personas, hacerlo poder ayuda...",[como sanarme crecer para ayudar a mi entorno ...
8,8,29,8_angel_guarda_angel guardian_guardian,"[angel, guarda, angel guardian, guardian, llam...","[llama angel guardian, guardian llama angel, a...","[quien es mi angel guardian, como se llama mi ..."
9,9,40,9_camino_camino correcto_correcto_voy,"[camino, camino correcto, correcto, voy, sueno...","[marcara camino voy, camino correcto, camino d...","[cual es mi camino, si lo que estoy estudiando..."


In [ ]:
Topics_k_means_repr_stop["KeyBERT"].head()

0    [ensene conectarme angeles, comunicarme angele...
1    [obtener sabiduria alcanzar, obtener sabiduria...
2    [claras ahora preguntar, pregunta, pregunta cl...
3    [mision objetivo vida, mision vida enfocarme, ...
4    [poder conectarme, podrian ayudarme, interesa ...
Name: KeyBERT, dtype: object

In [ ]:
topics_to_rake=', '.join(Topics_k_means_repr_stop["KeyBERT"].astype(str)) #llevamos a string

In [ ]:
topics_to_rake=topics_to_rake.replace('[','').replace(']','') #quitamos los simbolos innecesarios
topics_to_rake

"'ensene conectarme angeles', 'comunicarme angeles', 'conectarme angeles', 'terapeuta angelical', 'pequena aprenda angeles', 'conectar angeles', 'angelicales', 'angeles', 'pedirles guia sabiduria', 'pregunta aumentar fe', 'obtener sabiduria alcanzar', 'obtener sabiduria', 'persona amor sabiduria', 'obtener sabiduria abrir', 'sabiduria tener amor', 'obtener sabiduria bloqueo', 'unido obtener sabiduria', 'mas sabiduria', 'mundo obtener sabiduria', 'obtener verdadero conocimiento', 'claras ahora preguntar', 'pregunta', 'pregunta clara', 'preguntar pregunta clara', 'preguntar', 'preguntar pregunta', 'segura', 'ahora preguntar pregunta', 'ahora preguntar', 'bien', 'mision objetivo vida', 'mision vida enfocarme', 'mision vida requiero', 'mision vida seguir', 'reconocer mision vida', 'poner proposito vida', 'vida mision vida', 'mision mision vida', 'mision vida persona', 'preguntas mision vida', 'poder conectarme', 'podrian ayudarme', 'interesa saber conectar', 'mas debo poder', 'forma manten

In [ ]:
#?? spanish_tokenizer = nltk.data.load('tokenizers/punkt/PY3/spanish.pickle') #un tokenizador para RAKE

In [ ]:
r_new = Rake()
r_new.get_keywords_from_raw_text(topics_to_rake)
phrase_df_new = pd.DataFrame(r_new.get_keywords_with_scores(), columns = ['score','phrase'])

In [ ]:
res_1 =phrase_df_new.loc[phrase_df_new.score>5].sort_values('score', ascending = False)
res_1.shape[0]


431

In [ ]:
res_1 #RAKE on top de KeyBertInspired, 431 frase clave

,score,phrase
165,9.000000,ven hablan visualizarlos
16,9.000000,plenitud campo laboral
338,9.000000,hablando creadores diosa
262,9.000000,tantas empezar preguntas
351,9.000000,tendria muchas preguntas
...,...,...
82,5.011905,mismos angeles
263,5.011905,angeles escuchan
185,5.011905,contactar angeles
137,5.011905,angeles manifiestan


In [ ]:
#vemos que el score de las frases se baja bruscamente despues de los 7.5 approximadamente. Y los mas
#quitemos la parte que es mucho menos representativa
res_1_top =phrase_df_new.loc[phrase_df_new.score>8.8].sort_values('score', ascending = False)
res_1_top.shape[0]

34

In [ ]:
res_1_top

,score,phrase
260,9.000000,humanidad dios sienta
351,9.000000,tendria muchas preguntas
318,9.000000,perdoname pensado ello
262,9.000000,tantas empezar preguntas
261,9.000000,gente honesta ayudarles
16,9.000000,plenitud campo laboral
253,9.000000,humm realmente mos
329,9.000000,integral dos hijos
246,9.000000,gente conecten comprendan
332,9.000000,presentes despierten conciencia


##Resumen
Se ve que aun si se quitan las frases claves que se alejan por el score, no logramos hacer mas comprensible esta lista resultante para el ser humano.

Por tanto se opta por mostrar los N top scored frases, siendo N un numero que elegiria el usuario del algoritmo, dependiendo de que tan detallado quiere estudiar el tema.


SIn embargo, estas frases claves aun asi pueden ser nejoradas por modelos tipo Llama2 extraxtivos. Actuamente no es posible acceder a la msima en su version para español, debido a su tamaño, que aun con un procesamiento de optimizacion no ha sido posible descargar y trabajar con ello en el google collab de manera gratuita. Pero se sugiere buscar manera de utilizarla como uno de los modelos mas state of the art que estan entrenados a contestar preguntas, es decir - extraer informacion mas exacta del texto provisto y transformar la conclusion a un lenguaje humano bien compuesto.

In [ ]:
res_1.head(30)

,score,phrase
165,9.000000,ven hablan visualizarlos
16,9.000000,plenitud campo laboral
338,9.000000,hablando creadores diosa
262,9.000000,tantas empezar preguntas
351,9.000000,tendria muchas preguntas
260,9.000000,humanidad dios sienta
332,9.000000,presentes despierten conciencia
51,9.000000,controlar empiezo tantas
108,9.000000,hijos mio mismo
416,9.000000,primeros pasos puedes


In [ ]:
res_1.tail(20)

,score,phrase
392,5.115385,personas salud
296,5.115385,ayudar personas
74,5.115385,personas escuchada
354,5.114286,manera comunicarme
164,5.100000,trayectoria personal
2,5.100000,personal economicamente
371,5.098039,pedir angel
94,5.083333,ayuda angeles
6,5.071429,oportunidades ayuda
24,5.055556,ninguna daria


#Solo RAKE

In [ ]:
df_preguntas_tildes.head()

0                       cual es mi mision en esta vida
1    que debo hacer para estar en contacto con ello...
2       me pueden sanar de cualquier enfermedad fisica
3                                 como trabajar el ego
4                           cual es mi mision de vida?
Name: 8. Si estuvieras compartiendo un café con un terapeuta angelical, ¿Cuál sería la pregunta más importantes que le pedirías que te respondiera? - 1. Input - textarea, dtype: object

In [ ]:
#Quitando stopwords

In [ ]:
#stop = stopwords.words('spanish')
#stop=stop + ['si', 'tan', 'va', 'podria', 'hacer', 'pues', 'pasa', 'gustaria', 'tambien', 'demas',
             'ser', 'seria', 'serian', 'puedo', 'aun', 'sere', 'lograr', 'realizar',
              'hace', 'solo', 'aca', 'hacia','asi', 'deberia']
#df_solo_rake = df_preguntas_tildes.apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [ ]:
#De data frame  a lista + quitando simbolos innecesarios

In [ ]:
topics_to_rake2=df_preguntas_tildes.replace('[','').replace(']','')
topics_to_rake2 = ', '.join(df_preguntas_tildes)


In [ ]:
#aplicando rake
r_new = Rake()
r_new.get_keywords_from_raw_text(topics_to_rake2)
phrase_df_new2 = pd.DataFrame(r_new.get_keywords_with_scores(), columns = ['score','phrase'])

In [ ]:
phrase_df_new2.shape

(3119, 2)

In [ ]:
res_2 =phrase_df_new2.loc[phrase_df_new2.score>100].sort_values('score', ascending = False)
res_2 #solo RAKE

,score,phrase
2072,232.709094,como sabes que estas con una energia angelical...
2531,215.941828,en que estoy escaseando preguntaria acerca de ...
1054,205.247188,como minimizar mi ego por sobre todo para que ...
2604,198.016708,en cada una de las dimensiones materiales exis...
1676,196.018704,le preguntaria el porque uno tarda tanto en da...
...,...,...
2731,101.612115,los arcangeles los unicos que tienen la potest...
1664,101.082624,en este sentido le preguntaria sobre el bienes...
1789,100.630650,como dominar un ego tan fuerte que ese si es c...
2253,100.417699,ser una mejor guia para quienes ya partieron p...


In [ ]:
res_2.shape

(80, 2)

COMPARACION

In [ ]:
#Concatenamos resultados a un unico DF

In [ ]:
df_results = pd.concat([res_1["phrase"].reset_index(drop=True), res_2["phrase"].reset_index(drop=True) ], axis=1)
# Rename the columns
df_results.columns = ['BertRake', 'soloRake']
df_results

,BertRake,soloRake
0,ven hablan visualizarlos,como sabes que estas con una energia angelical...
1,plenitud campo laboral,en que estoy escaseando preguntaria acerca de ...
2,hablando creadores diosa,como minimizar mi ego por sobre todo para que ...
3,tantas empezar preguntas,en cada una de las dimensiones materiales exis...
4,tendria muchas preguntas,le preguntaria el porque uno tarda tanto en da...
...,...,...
426,mismos angeles,NaN
427,angeles escuchan,NaN
428,contactar angeles,NaN
429,angeles manifiestan,NaN


#KeyBERT with k-means on a text as a list

In [ ]:
from nltk.tokenize import sent_tokenize

text=df_preguntas_tildes.astype(str).to_list()
print(type(text))


<class 'list'>


In [ ]:
# KeyBERT
keybert = KeyBERTInspired()

# All representation models
representation_model = {
    "KeyBERT": keybert
}
cluster_model = KMeans(n_clusters=50)

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=None)

k_means_repr_stop = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                        vectorizer_model=vectorizer_model,
                        hdbscan_model=cluster_model,
                        umap_model = umap_model,
                        representation_model = representation_model,
                        min_topic_size=3,
                        nr_topics=50,
                        calculate_probabilities= True,
                        verbose=True )
topics, probabilities = k_means_repr_stop.fit_transform(text)
Topics_k_means_repr_stop=k_means_repr_stop.get_topic_info()
Topics_k_means_repr_stop

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

2023-10-04 00:14:44,558 - BERTopic - Transformed documents to Embeddings
2023-10-04 00:14:51,718 - BERTopic - Reduced dimensionality
2023-10-04 00:14:51,933 - BERTopic - Clustered reduced embeddings
2023-10-04 00:14:55,133 - BERTopic - Reduced number of topics from 50 to 50


,Topic,Count,Name,Representation,KeyBERT,Representative_Docs
0,0,66,0_sanar_salud_personas_sanarme,"[sanar, salud, personas, sanarme, como sanar, ...","[sanar otras personas, como sanar mi, como san...","[como ayudar a las personas, como sanar, que m..."
1,1,41,1_los_angeles_los angeles_comunicacion,"[los, angeles, los angeles, comunicacion, con,...","[angeles como comunicarme, los mensajes angeli...",[como conectar correctamente con los angeles y...
2,2,42,2_no se_no_se no_se,"[no se, no, se no, se, no se no, se no se, me ...","[no se, no se en, no lo se, ocurre no se, lo s...","[no se, no se, no se me ocurre]"
3,3,25,3_porque_no_he_que,"[porque, no, he, que, por que, que no, me, no ...","[porque mi vida, mi vida desde, por que estoy,...",[por que no he podido salir adelante que me pa...
4,4,27,4_escucharlos_escuchan_como_estan,"[escucharlos, escuchan, como, estan, me, me es...","[como escucharlos como, de escucharlos, como e...","[como saber que ellos.escuchan un pedido, como..."
5,5,47,5_mision_mi mision_vida_es mi mision,"[mision, mi mision, vida, es mi mision, es mi,...","[vida mi mision, mision de vida, es mi mision,...","[cual es mi mision en esta vida ?, cual es mi ..."
6,6,40,6_espiritual_como_con_para,"[espiritual, como, con, para, espirituales, ma...","[espiritualmente como puedo, ser mas espiritua...",[la mejor tecnica o forma para conectar con lo...
7,7,10,7_elevar_vibracion_mi frecuencia_elevar mi fre...,"[elevar, vibracion, mi frecuencia, elevar mi f...","[trabajar con energias, poder elevar la, mismo...",[esto es realmente posible y no tiene consecue...
8,8,65,8_angeles_con los angeles_con los_los angeles,"[angeles, con los angeles, con los, los angele...","[angeles que me, los angeles como, los angeles...","[?como conectarme con los angeles?, me gustari..."
9,9,28,9_mas alla_alla_no se_se,"[mas alla, alla, no se, se, hay, mas, no, prac...","[que hay mas, que encarne en, mas alla para, m...","[son muchas y son mas bien de la practica..., ..."


In [ ]:
Topics_k_means_repr_stop["KeyBERT"].shape

(50,)

In [ ]:
Topics_k_means_repr_stop["KeyBERT"][7]

['trabajar con energias',
 'poder elevar la',
 'mismo poder elevar',
 'puedo vibrar adecuadamente',
 'hacer para elevar',
 'si podrian trabajar',
 'mi vibracion alta',
 'mi frecuencia vibratoria',
 'mantener mi vibracion',
 'para elevar mi']

In [ ]:
df_results = pd.concat([res_1["phrase"].reset_index(drop=True), res_2["phrase"].reset_index(drop=True), Topics_k_means_repr_stop["KeyBERT"].reset_index(drop=True)  ], axis=1)
# Rename the columns
df_results.columns = ['KeyBERTRake', 'soloRake', 'KeyBERTList']
df_results

,KeyBERTRake,soloRake,KeyBERTList
0,ven hablan visualizarlos,como sabes que estas con una energia angelical...,"[sanar otras personas, como sanar mi, como san..."
1,plenitud campo laboral,en que estoy escaseando preguntaria acerca de ...,"[angeles como comunicarme, los mensajes angeli..."
2,hablando creadores diosa,como minimizar mi ego por sobre todo para que ...,"[no se, no se en, no lo se, ocurre no se, lo s..."
3,tantas empezar preguntas,en cada una de las dimensiones materiales exis...,"[porque mi vida, mi vida desde, por que estoy,..."
4,tendria muchas preguntas,le preguntaria el porque uno tarda tanto en da...,"[como escucharlos como, de escucharlos, como e..."
...,...,...,...
426,mismos angeles,NaN,NaN
427,angeles escuchan,NaN,NaN
428,contactar angeles,NaN,NaN
429,angeles manifiestan,NaN,NaN
